In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = "/content/drive/My Drive/mini_project/dataset/cleaned/"
img_size = 100

In [ ]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('/content/drive/My Drive/mini_project/dataset/cleaned')

In [ ]:
classes_images_dict = {
    'carrying': list(data_dir.glob('carrying/*')),
    'threat': list(data_dir.glob('threat/*')),
    'normal': list(data_dir.glob('normal/*')),
}

In [ ]:
classes_labels_dict = {
    'carrying': 0,
    'threat': 1,
    'normal': 2,
}

In [ ]:
X, y = [], []

for class_name, images in classes_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(img_size,img_size))
        X.append(resized_img)
        y.append(classes_labels_dict[class_name])

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)


In [25]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [26]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_size, 
                                                              img_size,
                                                              3)
                                                              ),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1)
  ]
)

In [27]:
num_classes = 3

model = Sequential([
  data_augmentation,
    
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
    
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
    
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),

  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.4),
  
  layers.Flatten(),
    
  layers.Dense(64, activation='relu'),
  layers.Dense(num_classes)
])
 

In [29]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
m = model.fit(X_train_scaled, y_train, epochs=150)

Epoch 1/150


KeyboardInterrupt: ignored

In [ ]:
model.evaluate(X_test_scaled,y_test)

In [ ]:
history = m

In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

pred = model.predict(X_test_scaled)

classes = ["carrying", "threat", "normal"]

cm = confusion_matrix(y_test, pred.argmax(axis=1))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)

disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef

print(f1_score(y_test, pred.argmax(axis=1), average='macro'))
print(precision_score(y_test, pred.argmax(axis=1), average='macro'))
print(recall_score(y_test, pred.argmax(axis=1), average='macro'))

print(matthews_corrcoef(y_test, pred.argmax(axis=1)))


In [ ]:
model.evaluate(X_train_scaled,y_train)